# Quais são as universidades de excelência no Brasil, segundo os dados do Lieden Ranking?

O objetivo desta análise é encontrar centros de excelência em pesquisa nas universidades brasileiras. Entretanto, definir **excelência universiária** não é tarefa simples. 

Usualmente, as avaliações acadêmicas usam critérios de impacto científico – ou seja, uma avaliação quantitativa de publicações e citações – para determinar quais são os programas e instituições de destaque. O uso dsse tipo de métrica, porém, é questionado por pesquisadores e cientometristas por carregar uma série de viéses.

Usar uma contagem simples do número de artigos publicados ou citações para definir quais instituições estão fazendo um bom trabalho pode, por exemplo, destacar apenas as universidades que concentram suas pesquisas em áreas onde, pela própria natureza do campo, costuma-se publicar e citar mais. Ciências exatas e biológicas, nesse caso, seriam privilegiadas em detrimento das ciênciais sociais e humanas.

Esse problema ocorre também dentro de um mesmo campo científico: pesquisas sobre câncer, por exemplo, tendem a ser mais citadas que pesquisas sobre doenças tropicais, ainda que estas últimas sejam muito relevantes para a saúde brasileira.

Para minimizar esses problemas, optamos por usar os dados do [Lieden Ranking](https://www.leidenranking.com) como ponto de início para nossa reportagem. Eles não oferecem apenas um número mágico, como outros rankings universitários, mas sim indicadores de impacto e produtividade segmentados por campo do conhecimento. Além disso, trazem outros dados de interesse, como paridade de gênero entre pesquisadores, quantidade de artigos publicados em portais de acesso aberto e métricas de colaboração entre instituições de países diferentes. Com isso, podemos olhar para diferentes aspectos na hora de procurar universidades que se destacam.

Com os dados em mãos, vamos procurar instituições com maior impacto nas [cinco áreas do conhecimento identificadas pelo ranking](https://www.leidenranking.com/information/fields), usando tanto uma contagem absoluta de artigos quanto números relativos. Além disso, também queremos encontrar as universidades que se destacam nos critérios de **colaboração internacional**, **publicação de artigos de acesso aberto** e **paridade de gênero**.


## 1. Importar pacotes

In [1]:
# Importa o pacote pandas, que implementa funções de análise de dados
import pandas as pd

## 2. Acessar dados

In [2]:
# Lê a aba "Results" do arquivo Excel com os dados do ranking
with pd.ExcelFile("../data/CWTS Leiden Ranking 2019.xlsx") as xls:
    df = pd.read_excel(xls, 'Results')

## 3. Descrição e seleção dos campos de interesse

Estes são os campos do banco de dados.

In [3]:
# Exibe as colunas do banco de dados
df.columns

Index(['University', 'Country', 'Field', 'Period', 'Frac_counting', 'impact_P',
       'collab_P', 'oa_P', 'gender_A', 'gender_A_MF', 'TCS', 'TNCS', 'P_top1',
       'P_top5', 'P_top10', 'P_top50', 'P_collab', 'P_int_collab',
       'P_industry_collab', 'P_short_dist_collab', 'P_long_dist_collab',
       'P_OA', 'P_gold_OA', 'P_hybrid_OA', 'P_bronze_OA', 'P_green_OA',
       'P_OA_unknown', 'A_gender_unknown', 'A_M', 'A_F', 'MCS', 'MCS_lb',
       'MCS_ub', 'MNCS', 'MNCS_lb', 'MNCS_ub', 'PP_top1', 'PP_top1_lb',
       'PP_top1_ub', 'PP_top5', 'PP_top5_lb', 'PP_top5_ub', 'PP_top10',
       'PP_top10_lb', 'PP_top10_ub', 'PP_top50', 'PP_top50_lb', 'PP_top50_ub',
       'PP_collab', 'PP_collab_lb', 'PP_collab_ub', 'PP_int_collab',
       'PP_int_collab_lb', 'PP_int_collab_ub', 'PP_industry_collab',
       'PP_industry_collab_lb', 'PP_industry_collab_ub',
       'PP_short_dist_collab', 'PP_short_dist_collab_lb',
       'PP_short_dist_collab_ub', 'PP_long_dist_collab',
       'PP_long_dist_c

Para esta reportagem, vamos olhar para os seguintes campos:

- `impact_P`: total de publicações de uma universidade.
- `P_top10` e `PP_top10`: respectivamente, o número e a proporção de publicações de uma universidade que, em comparação com outras publicações no mesmo campo e no mesmo ano, pertencem aos 10% mais frequentemente citados.
- `P_int_collab` e `PP_int_collab`: respectivamente, o número e a proporção de publicações de uma universidade que foram feitos em co-autoria entre dois ou mais países.
- `P_industry_collab` e `PP_industry_collab`: respectivamente, o número e a proporção de publicações de uma universidade que foram feitos em co-autoria com uma instituição do setor privado (que não sejam instituições de ensino ou hospitais)
- `P_OA` e `PP_OA`: respectivamente, o número e a proporção das publicações de acesso aberto de uma universidade.
- `A_F`, `PA_F`, and `PA_F_MF`): respectivamente, o número de autores mulheres nas publicações da universidade, o percentual de autores mulheres em relação ao total de autores de uma universidade (incluindo aqueles de gênero desconhecido) e o percentual de autores mulheres em relação ao total de autores homens e mulheres da universidade.

Também devem ser considerados os indicadores `TNCS` e `MNCS`, que representam o total de citações de citações e a média de citações de uma universidade, mas que são **normalizados** em relação ao total de citações do ano em que os artigos foram publicados e do campo de conhecimento em que a pesquisa se insere.

Uma descrição completa do significado dos indicadores está disponível [aqui](https://www.leidenranking.com/information/indicators).

In [4]:
# Mantém apenas as colunas relevantes. No caso, os campos identificadores e os indicadores citados acima.
df = df [[ "University", "Country", "Field", "Period", "Frac_counting", 
           "impact_P", "P_top10", "PP_top10", "P_int_collab", "PP_int_collab",
           "P_industry_collab", "PP_industry_collab", "P_OA", "PP_OA", 
           "A_F", "PA_F", "PA_F_MF" ]]

**Sobre o parâmetro `Frac_counting`:**

O parâmetro contagem fracional, quando ativo, dá peso menor para o impacto de artigos publicados colaborativamente entre várias universidades. 

Exemplo: um artigo foi publicado por cinco pesquisadores, dois destes da USP e três de Harvard. A publicação vai fazer com que a USP some 0.4 pontos (2/5) e Harvard 0.6 pontos (3/5) no indicador `impact_P` e em todos os indicadores derivados.

De acordo com as orientações [na documentação do próprio Lieden Ranking](https://www.leidenranking.com/information/indicators#counting-method), vamos avaliar os dados com o parâmetro **ativo**.

**É importante frisar que o fractional counting vale para os indicadores de impacto, mas não para os indicadores de acesso aberto, colaboração e gênero.**

In [5]:
# Mantém apenas as linhas nas quais o parâmetro Frac_counting está ativo
df = df [ df.Frac_counting == 1]

**Sobre o intervalo de tempo selecionado:**

Para evitar que aumentos ou quedas aleatórias na produtividade afetem os indicadores de qualidade universitária, o Lieden Ranking agrupa os dados em janelas de quatro anos. Assim, um ano atípico não faz com que as métricas de uma instituição disparem ou despenquem.

Como queremos olhar para o estado atual das universidades brasileiras, vamos selecionar os dados do recorte mais recente possível: 2014 – 2017.

Há, é verdade, uma defasagem de quase três anos entre a publicação dos dados e a produção da reportagem. Entretanto, estes são os dados mais recentes possíveis. Pode ser que a realidade das universidades tenha se alterado desde então? Sim, pode, mas nossa análise não vai se basear apenas nos números. Ela engloba, também, esforço de reportagem.

In [6]:
# Mantém apenas as linhas nas quais o parâmetro Period é o intervalo entre 2014 e o 2017
df = df [ df.Period == "2014–2017"]

Para facilitar o processamento, serão mantidas apenas as linhas referentes a universidades brasileiras.

In [7]:
# Mantém apenas as linhas em que o país é o Brasil
df = df [ df.Country == "Brazil" ]

## 4. Destaques em números absolutos

De início, vamos analisar as variáveis `impact_P` e `P_top10`. Assim, estaremos olhando para a **quantidade absoluta de artigos publicados** e a **quantidade absoluta de artigos entre os 10% mais citados do respectivo campo do conhecimento**.

### 4.1 Universidades que se destacam no agregado dos campos do conhecimento

De início, vamos avaliar quais universidades **publicam mais artigos nos periódicos considerados pelo ranking** (campo `Impact_P`), sem discriminar por campo do conhecimento.

In [8]:
# Ordena o banco de dados pelo valor na coluna 'impact_P', de forma decrescente. Depois, exibe as primeiras cinco linhas.
df.sort_values(by="impact_P", ascending = False).head(10)

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
89333,University of São Paulo,Brazil,All sciences,2014–2017,1,16845.891826,1050.728503,0.062373,14757.000000,0.443220,819.000000,0.024598,13694.000000,0.411293,39490.000000,0.383737,0.419179
89351,University of São Paulo,Brazil,Biomedical and health sciences,2014–2017,1,8391.493343,495.994961,0.059107,6295.500000,0.394942,438.333333,0.027498,7197.166667,0.451507,26547.333333,0.450965,0.476457
61793,Universidade Estadual Paulista,Brazil,All sciences,2014–2017,1,6324.775011,333.787283,0.052775,4600.000000,0.372470,156.000000,0.012632,4661.000000,0.377409,15553.000000,0.397968,0.437817
72161,University of Campinas,Brazil,All sciences,2014–2017,1,5562.043446,368.069548,0.066175,4316.000000,0.381104,256.000000,0.022605,4073.000000,0.359647,12071.000000,0.381692,0.418130
15245,Federal University of Rio Grande do Sul,Brazil,All sciences,2014–2017,1,5372.214621,324.863294,0.060471,3707.000000,0.382717,240.000000,0.024778,3272.000000,0.337807,15098.000000,0.467923,0.494789
62765,Universidade Federal do Rio de Janeiro,Brazil,All sciences,2014–2017,1,4780.845925,311.112211,0.065075,4649.000000,0.444455,423.000000,0.040440,4458.000000,0.426195,12191.000000,0.361633,0.445920
62117,Universidade Federal de Minas Gerais,Brazil,All sciences,2014–2017,1,4121.372963,252.693862,0.061313,3096.000000,0.386372,188.000000,0.023462,3079.000000,0.384251,11568.000000,0.441982,0.469442
89369,University of São Paulo,Brazil,Life and earth sciences,2014–2017,1,3702.895717,262.061184,0.070772,3442.000000,0.456640,192.833333,0.025583,2554.166667,0.338854,7814.333333,0.383112,0.406122
89405,University of São Paulo,Brazil,Physical sciences and engineering,2014–2017,1,3247.696944,196.780312,0.060591,3773.333333,0.528663,146.833333,0.020572,3075.166667,0.430846,3813.666667,0.205859,0.262009
62333,Universidade Federal de São Paulo,Brazil,All sciences,2014–2017,1,2991.923594,135.530425,0.045299,2347.000000,0.357993,195.000000,0.029744,2986.000000,0.455461,9966.000000,0.484633,0.510396


Já de início, uma supresa. **Se isolássemos apenas as publicações da área de Ciências Biomédicas e da Saúde da USP, teríamos a segunda universidade mais produtiva do país** – atrás apenas da própria USP.

A USP está **muito** acima das demais universidades em volume de publicações. O indicar `impact_P` é duas vezes maior que o da segunda universidade que mais publica, a Unesp.

Depois, aparecem outras universidades do sistema estadual paulista: Unesp e Unicamp.

Estamos olhando, entretando, para o total de artigos publicados em revistas internacionais. Podemos limitar o recorte e analisar apenas as publicações que atingiram maior nível de impacto: o indicador `P_top10` só apenas as publicações que estão entre as 10% mais citadas de suas respectivas áreas em um determinado ano.

In [9]:
# Ordena os dados de forma decrescente de acordo com a coluna 'P_top10' e exibe as cinco primeiras linhas
df.sort_values(by="P_top10", ascending = False).head()

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
89333,University of São Paulo,Brazil,All sciences,2014–2017,1,16845.891826,1050.728503,0.062373,14757.0,0.443220,819.000000,0.024598,13694.000000,0.411293,39490.000000,0.383737,0.419179
89351,University of São Paulo,Brazil,Biomedical and health sciences,2014–2017,1,8391.493343,495.994961,0.059107,6295.5,0.394942,438.333333,0.027498,7197.166667,0.451507,26547.333333,0.450965,0.476457
72161,University of Campinas,Brazil,All sciences,2014–2017,1,5562.043446,368.069548,0.066175,4316.0,0.381104,256.000000,0.022605,4073.000000,0.359647,12071.000000,0.381692,0.418130
61793,Universidade Estadual Paulista,Brazil,All sciences,2014–2017,1,6324.775011,333.787283,0.052775,4600.0,0.372470,156.000000,0.012632,4661.000000,0.377409,15553.000000,0.397968,0.437817
15245,Federal University of Rio Grande do Sul,Brazil,All sciences,2014–2017,1,5372.214621,324.863294,0.060471,3707.0,0.382717,240.000000,0.024778,3272.000000,0.337807,15098.000000,0.467923,0.494789


As universidades de destaque se mantém praticamente na mesma ordem, inclusive em relação às pesquisas em saúde na USP. Isso significa que a mesma proporção de artigos atinge alto impacto. A única mudança de ordem em relação ao recorte anterior é que a Unicamp supera a Unesp.

### 4.2 Universidades que se destacam em campos do conhecimento específicos

Agora, vamos repetir essa mesma análise, mas separando as áreas de conhecimento. Assim, é possível descobrir que universidades mais produzem em cada área.
Para isso, precisamos definir antes algumas variáveis e funções.

In [10]:
# Seleciona os possíveis valoes da coluna 'Field' e guarda as entradas em uma lista
fields = df.Field.unique().tolist()

In [11]:
# Remove o item de valor 'All sciences' da lista acima
fields.remove('All sciences')

In [12]:
# Abaixo, vamos definir uma função que ordena o dataframe de acordo com um campo específico e
# imprime, lado a lado, o nome das cinco primeiras universidades e o valor da respectiva coluna.
def top_5(data, field, indicator):
    '''
    De início, a função filtra o o dataframe `data` para
    manter apenas as linhas em que a coluna "Field" seja igual
    ao especificado no parâmetro `field`. Depos, ordena o mesmo 
    de acordo com o campo especificado no parâmetro `indicator`. 
    Finalmente, imprime lado a lado, o nome das  cinco primeiras 
    universidades e o valor do indicador.
    
    Parâmetros:
    
    data -> Um dataframe do pandas
    field -> Uma string que representa qual valor da coluna
    "Field" deve ser mantido na hora de filtrar o dataframe `data`.
    indicator -> Uma string que representa o nome 
    de uma coluna do dataframe acima
    '''
    
    # Mantém apenas as linhas nas quais o campo do conhecimento
    # (coluna "Field") é igual ao especificado no parâmetro `field`
    data = data [ data.Field == field]
    
    # Ordena o dataframe de forma decrescente, de acordo com
    # o indicador especificado em `indicator`
    data = data.sort_values(by = indicator, ascending = False)

    # Mantém apenas as cinco primeiras linhas do dataframe
    data = data.head(5)
    
    # Imprime o nome do campo para facilitar a leitura
    # caso a função seja executada dentro de um loop
    
    print("#" * (len(field) + 4) )
    print("# " + field + " #")
    print("#" * (len(field) + 4) )
    print()
    
    # Define e imprime um cabeçalho para facilitar
    # a leitura do output
    header = "University".ljust(50) + f"{indicator}".ljust(10) # O método ljust(n) preenche 
    print(header)                                              # a string com whitespaces até 
    print()                                                    # que ela tenha comprimento igual a n.
    
    # Para cada uma das linhas do dataframe `data`,
    # imprime o nome da universidade e o valor
    # da coluna `indicator`, ladoa  lado.
    for index, row in data.iterrows():
        
        line = f"{row.University}".ljust(50) + f"{row[indicator]}".ljust(10)
        print(line)
        
    # Por fim, imprime um espaço para que o output não se sobreponha
    print()

Usando a função e a lista definidas acima, podemos facilmente descobrir quais são as universidades que mais produzem em cada campo.

In [13]:
# Chama a função uma vez para cada item na lsita, alterando o parâmetro 'field' 
for field in fields:
    top_5(df, field, "impact_P")

##################################
# Biomedical and health sciences #
##################################

University                                        impact_P  

University of São Paulo                           8391.49334315334
Universidade Estadual Paulista                    2624.41545708
Universidade Federal de São Paulo                 2410.04300561333
University of Campinas                            2292.88941949167
Federal University of Rio Grande do Sul           2259.34691314167

###########################
# Life and earth sciences #
###########################

University                                        impact_P  

University of São Paulo                           3702.89571653333
Universidade Estadual Paulista                    2042.18546828666
Federal University of Rio Grande do Sul           1331.526214505
Universidade Federal do Rio de Janeiro            1147.59083336667
University of Campinas                            1057.3624877

######################

Como é de se esperar, a USP lidera em todos os campos – ela é, afinal, disparadamente a maior universidade brasileira.

Entretanto, há instituições que não estão entre as cinco que mais publicam artigos como um todo, mas que se destacam em campos específicos.

A **Unifesp**, por exemplo, aparece entre as cinco primeiras na área de saúde. A **UFRJ** se destaca na área de Ciências da Terra.

 Em matemática e ciência da computação, destacam-se **UFRJ**, **UFPE** e **UFMG**.
 
 Em Ciências físicas, a **UFRJ** aparece bem posicionada de novo.
 
 Finalmente, em ciências sociais e humanidades, a lista de cinco melhores  a **UFRGS** aparece em segundo, **UFRJ** e **UFMG** aparecem novamente e a **UnB** aparece pela primeira vez como destaque.
    

Agora, vamos ver quais são os resultados usando a métrica `P_top10` – ou seja, considerando apenas artigos que tenham atingido alto impacto.

In [14]:
for field in fields:
    top_5(df, field, "P_top10")

##################################
# Biomedical and health sciences #
##################################

University                                        P_top10   

University of São Paulo                           495.994960989405
Federal University of Rio Grande do Sul           127.164453106562
Universidade Estadual Paulista                    122.190380979349
University of Campinas                            112.737921966259
Universidade Federal do Rio de Janeiro            112.282481903485

###########################
# Life and earth sciences #
###########################

University                                        P_top10   

University of São Paulo                           262.061183719962
Universidade Estadual Paulista                    107.054815179322
University of Campinas                            91.8348607364319
Federal University of Rio Grande do Sul           86.3774061302767
Universidade Federal do Rio de Janeiro            79.7845649329967

#############

A maior parte dos resultados anteriores se mantém, com algumas alterações pontuais. Algo que chama a atenção, de imediato, é o avanço da **UFRGS** na área de ciências da saúde. 

Esses números, porém, seguem refletindo em grande medida o **tamanho das universidades** – quem se destaca é, quase sempre, uma instituição do sistema estadual paulista ou que está entre as maiores universidades federais. O próximo indicador pode minimizar esse problema.

## 5. Destaques em números relativos

Agora, vamos analisar os dados usando o recorte `PP_top10`: o percentual de artigos publicados por cada universidade que está entre os 10% mais citados na respectiva área e ano.

Esse indicador mostra, grosso modo, o **aproveitamento** dos artigos publicados por cada universidade, revelando quem consegue emplacar mais estudos de alta relevância, em termos relativos.

### 5.1 Universidades que se destacam no agregado dos campos do conhecimento

Como na primeira sessão da análise, vamos começar olhando o desempenho das universidades no agregado de todos os campos do conhecimento.

In [15]:
df [ df.Field == "All sciences" ].sort_values(by = "PP_top10", ascending = False).head()

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
62549,Universidade Federal do Ceará,Brazil,All sciences,2014–2017,1,1629.036607,115.290766,0.070772,1299.0,0.391973,68.0,0.020519,1084.0,0.327097,4166.0,0.372130,0.405727
61901,Universidade Federal da Bahia,Brazil,All sciences,2014–2017,1,1155.601175,77.911443,0.067421,1005.0,0.380106,62.0,0.023449,1024.0,0.387292,2955.0,0.420760,0.459851
15461,Federal University of São Carlos,Brazil,All sciences,2014–2017,1,1838.897696,122.366391,0.066543,1419.0,0.365345,74.0,0.019053,1155.0,0.297374,3308.0,0.353721,0.386223
62225,Universidade Federal de Santa Catarina,Brazil,All sciences,2014–2017,1,2596.592015,172.235697,0.066331,2032.0,0.409595,139.0,0.028019,1383.0,0.278774,5448.0,0.387979,0.418884
72161,University of Campinas,Brazil,All sciences,2014–2017,1,5562.043446,368.069548,0.066175,4316.0,0.381104,256.0,0.022605,4073.0,0.359647,12071.0,0.381692,0.418130


A lista de instituições que se destacam sob esse critério é bem diferente das anteriores. Vemos universidades menores, como a **UFC**, a **UFBA**, a **Ufscar** e a **UFSC** na liderança. 

Todas elas têm um percentual semelhante de artigos de destaque: entre 6,5 e 7%.

Para saber se isso é relevante, precisamos comparar com os números das instituições que listamos no primeiro item deste caderno.

In [16]:
# Salva o nome das cinco universidades com maior indicador 'impact_P' em uma lista
top_5_unis_by_impact_p = df.sort_values(by="impact_P", ascending = False).head(5).University.tolist()

In [17]:
# Seleciona as universidades que estão na lista definida acima, 
# mantém as linhas em que o campo Field é "All sciences",
# e ordenas as linhas do maior PP_top10 para o menor
df [ df.University.isin(top_5_unis_by_impact_p) & (df.Field == "All sciences") ].sort_values(by = "PP_top10", ascending = False)

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
72161,University of Campinas,Brazil,All sciences,2014–2017,1,5562.043446,368.069548,0.066175,4316.0,0.381104,256.0,0.022605,4073.0,0.359647,12071.0,0.381692,0.418130
89333,University of São Paulo,Brazil,All sciences,2014–2017,1,16845.891826,1050.728503,0.062373,14757.0,0.443220,819.0,0.024598,13694.0,0.411293,39490.0,0.383737,0.419179
15245,Federal University of Rio Grande do Sul,Brazil,All sciences,2014–2017,1,5372.214621,324.863294,0.060471,3707.0,0.382717,240.0,0.024778,3272.0,0.337807,15098.0,0.467923,0.494789
61793,Universidade Estadual Paulista,Brazil,All sciences,2014–2017,1,6324.775011,333.787283,0.052775,4600.0,0.372470,156.0,0.012632,4661.0,0.377409,15553.0,0.397968,0.437817


Há diferenças relevantes no percentual. A **Unicamp** tem um desempenho parecido com as universidades listadas acima, mas as instituições que estão abaixo dela já mostram um percentual relativamente menor. Na Unesp, por exemplo, o percentual é de 5.2% – cerca de um ponto perentual a menos do que a média do grupo destacado anteriormente.

### 5.2 Universidades que se destacam em campos do conhecimento específicos

Vamos repetir a análise recorte acima, mas dessa vez segmentando os dados por área do conhecimento.

In [18]:
for field in fields:
    top_5(df, field, "PP_top10")

##################################
# Biomedical and health sciences #
##################################

University                                        PP_top10  

Federal University of Uberlândia                  0.0679811092919161
Universidade Federal do Rio de Janeiro            0.0645691142429298
Universidade Federal da Bahia                     0.0619537705143072
Universidade Federal de Santa Catarina            0.061688635513299
University of São Paulo                           0.0591068765363544

###########################
# Life and earth sciences #
###########################

University                                        PP_top10  

University of Campinas                            0.0868527697972276
Universidade Federal de São Paulo                 0.084795353981237
Universidade Federal de Santa Catarina            0.0712151009667014
University of São Paulo                           0.0707719589698047
Universidade Federal de Minas Gerais              0.0699860862449

Perceba que o grupo de cinco unidades de destaque varia muito de acordo com a área do conhecimento que escolhemos. Isso revela que é difícil que uma universidade seja extremamente produtiva em todas as áreas. Assim, faz sentido destacar quem vai melhor em cada campo. 

Com base nos resultados acima, podemos elaborar uma lista de pesquisadores e departamentos para entrevistar consultando o banco de dados do *Web of Science*.

## 6. Universidades que mais produzem artigos em acesso aberto

Todos os recortes acima levam em consideração a **quantidade de artigos publicados** ou a **quantidade de citações** de cada universidade. Há, entretanto, outras métricas possíveis. A partir de agora, vamos olhar não apenas para o **impacto**, mas para as **características** da pesquisa de cada universidade.

De início, vamos destacar as universidades com **mais artigos de acesso aberto, percentualmente** (`PP_OA`) – ou seja, artigos publicados em periódicos que podem ser acessados de forma livre.

### 6.1 Análise agregada

Primeiro, vamos avaliar a produção aberta de cada universidade, na soma de todos os campos do conhecimento.

In [19]:
df [ df.Field == "All sciences" ].sort_values(by = "PP_OA", ascending = False).head(5)

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
62333,Universidade Federal de São Paulo,Brazil,All sciences,2014–2017,1,2991.923594,135.530425,0.045299,2347.0,0.357993,195.0,0.029744,2986.0,0.455461,9966.0,0.484633,0.510396
48077,Rio de Janeiro State University,Brazil,All sciences,2014–2017,1,1227.712649,52.988689,0.043160,1307.0,0.411524,73.0,0.022985,1442.0,0.454030,3253.0,0.236204,0.442164
62765,Universidade Federal do Rio de Janeiro,Brazil,All sciences,2014–2017,1,4780.845925,311.112211,0.065075,4649.0,0.444455,423.0,0.040440,4458.0,0.426195,12191.0,0.361633,0.445920
15137,Federal University of Rio Grande do Norte,Brazil,All sciences,2014–2017,1,1439.328069,71.700217,0.049815,1159.0,0.393816,51.0,0.017329,1248.0,0.424057,3049.0,0.374939,0.425066
14921,Federal University of Paraíba,Brazil,All sciences,2014–2017,1,1067.786027,67.484676,0.063201,738.0,0.319342,25.0,0.010818,979.0,0.423626,1815.0,0.318757,0.363582


A **Unifesp** não é apenas uma das universidades que mais publicam no país, mas é a que mais cria conteúdo de acesso aberto: 45.6% de suas publicações são livres. Em segundo, porém, vem a **UERJ** (45.4%). UFRJ, UFRN e UFPB vêm em seguida, com um indicador próximo de 42%.

### 6.2 Análise por campo do conhecimento

Será que as publicações abertas dessas universidades se concentram em algum campo específico? Vamos verificar segmentando o recorte por áreas do conhecimento.

In [20]:
for field in fields:
    top_5(df, field, "PP_OA")

##################################
# Biomedical and health sciences #
##################################

University                                        PP_OA     

Universidade Federal do Rio de Janeiro            0.491830985915493
Universidade Federal da Bahia                     0.488281856278131
Federal University of Rio Grande do Norte         0.482740943267259
Universidade Federal de São Paulo                 0.478839863827515
Universidade de Brasília                          0.465950284826515

###########################
# Life and earth sciences #
###########################

University                                        PP_OA     

Universidade Federal de São Paulo                 0.405367231638418
Universidade Federal de Minas Gerais              0.374498439589835
Universidade Federal de Goiás                     0.355905818252578
Federal University of Uberlândia                  0.343065693430657
University of São Paulo                           0.338853756688631

###

A prudução de acesso aberto da **UERJ**, que já apareceu com destaque na soma de todos os campos do conhecimento, se destaca ainda mais quando segmentamos os resultados por área. 65.6% do conteúdo publicado na área de *Physical sciences and engineering* é livre.

## 7. Universidades que mais colaboram com universidades estrangeiras

Outra métrica interessante é quantas pesquisas a universidade realizou em colaboração com instituições estrangeiras. Vamos, novamente, usar a medida em percentual (`PP_int_collab`). 

### 7.1 Análise agregada

De início, como usual, vamos ver o desempenho de cada universidade no agregado dos campos do conhecimento.

In [21]:
df [ df.Field == "All sciences" ].sort_values(by = "PP_int_collab", ascending = False).head(5)

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
62765,Universidade Federal do Rio de Janeiro,Brazil,All sciences,2014–2017,1,4780.845925,311.112211,0.065075,4649.0,0.444455,423.0,0.040440,4458.0,0.426195,12191.0,0.361633,0.445920
89333,University of São Paulo,Brazil,All sciences,2014–2017,1,16845.891826,1050.728503,0.062373,14757.0,0.443220,819.0,0.024598,13694.0,0.411293,39490.0,0.383737,0.419179
61577,Universidade de Brasília,Brazil,All sciences,2014–2017,1,1761.322343,92.952347,0.052774,1673.0,0.437043,97.0,0.025340,1398.0,0.365204,3259.0,0.348184,0.384452
48077,Rio de Janeiro State University,Brazil,All sciences,2014–2017,1,1227.712649,52.988689,0.043160,1307.0,0.411524,73.0,0.022985,1442.0,0.454030,3253.0,0.236204,0.442164
62225,Universidade Federal de Santa Catarina,Brazil,All sciences,2014–2017,1,2596.592015,172.235697,0.066331,2032.0,0.409595,139.0,0.028019,1383.0,0.278774,5448.0,0.387979,0.418884


**UFRJ**, **USP**, **UnB**, **UERJ** e **UFSC** aparecem em posições de destaque.

Para termos melhor referência, vamos olhar também para a média de colaboração nas universidades do país.

In [22]:
df.PP_int_collab.mean()

0.3886358914364735

### 7.2 Análise por campo do conhecimento

Agora, vamos repetir o recorte, mas olhando para campos do conhecimento individuais.

In [23]:
for field in fields:
    top_5(df, field, "PP_int_collab")

##################################
# Biomedical and health sciences #
##################################

University                                        PP_int_collab

Universidade Federal de Santa Catarina            0.41505376344086
Universidade de Brasília                          0.395649922320041
University of São Paulo                           0.39494155287426
Universidade Federal do Rio de Janeiro            0.392762730227519
Universidade Federal da Bahia                     0.387862796833773

###########################
# Life and earth sciences #
###########################

University                                        PP_int_collab

Universidade de Brasília                          0.457270319252753
University of São Paulo                           0.456639985848849
Federal University of Pernambuco                  0.434232954545454
Universidade Federal Fluminense                   0.420521653543307
Universidade Federal de Minas Gerais              0.417298261257245


O segmento de *Social sciences and humanities* da **UFPB** tem uma quantidade assustadora de pesquisas em colaboração: 71%. Vale o destaque.

## 8. Universidades com mais mulheres pesquisadoras

A participação de mulheres no cenário acadêmico também é um indicador de interesse. Podemos ver quem se destaca nesse aspecto usando o indicador `PA_F_MF`, que calcula o percentual de autores mulheres sobre o **percentual de autores homens e mulheres**. Isso quer dizer que a conta ignora os pesquisadores com gênero desconhecido ou ambíguo. 

Essas incertezas surgem porque a classificação feita pelo Lieden Ranking é automatizada. O sistema usa pacotes de programação que reconhecem e associam nomes comuns a um respectivo gênero. Assim, nomes incomuns ou que podem servir tanto para homens quanto melhores são ignorados pelo indicador selecionado.

### 8.1 Análise agregada

Primeiro, assim como antes, vamos ver o desempenho das universidades na soma de todas as áreas do conhecimento.

In [24]:
df [ df.Field == "All sciences" ].sort_values(by = "PA_F_MF", ascending = False).head(5)

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
54341,State University of Maringá,Brazil,All sciences,2014–2017,1,1288.118676,76.377662,0.059294,633.0,0.281333,34.0,0.015111,682.0,0.303111,4254.0,0.507456,0.541153
62333,Universidade Federal de São Paulo,Brazil,All sciences,2014–2017,1,2991.923594,135.530425,0.045299,2347.0,0.357993,195.0,0.029744,2986.0,0.455461,9966.0,0.484633,0.510396
15353,Federal University of Santa Maria,Brazil,All sciences,2014–2017,1,1666.459169,86.083091,0.051656,895.0,0.301550,46.0,0.015499,709.0,0.238881,5671.0,0.477638,0.505122
15245,Federal University of Rio Grande do Sul,Brazil,All sciences,2014–2017,1,5372.214621,324.863294,0.060471,3707.0,0.382717,240.0,0.024778,3272.0,0.337807,15098.0,0.467923,0.494789
62117,Universidade Federal de Minas Gerais,Brazil,All sciences,2014–2017,1,4121.372963,252.693862,0.061313,3096.0,0.386372,188.0,0.023462,3079.0,0.384251,11568.0,0.441982,0.469442


**UEM**, **USP** e **UFSM** são universidades que têm mais de 50% de mulheres autoras – situação próxima à paridade de gênero.

Para colocar esse dado em contexto, vamos ver também qual é a média de todas as universidades brasileiras no ranking.

In [25]:
for field in df.Field.unique():
    print(field, "–", df [ df.Field == field ]["PA_F_MF"].mean())

All sciences – 0.4368138203462944
Biomedical and health sciences – 0.5304707702358401
Life and earth sciences – 0.44959380498193724
Mathematics and computer science – 0.1764498007498029
Physical sciences and engineering – 0.3142433260137763
Social sciences and humanities – 0.40702180584376335


Na média, mulheres são 43.6% dos autores nas universidades. As três instituições citadas acima estão de fato bastante acima das demais.

### 8.2 Análise por campo do conhecimento

Vamos ver a quantidade de mulheres pesquisadoras em cada área do conhecimento, também.

In [26]:
for field in fields:
    top_5(df, field, "PA_F_MF")

##################################
# Biomedical and health sciences #
##################################

University                                        PA_F_MF   

State University of Maringá                       0.635043507740988
Federal University of Santa Maria                 0.609436844663565
Federal University of Rio Grande do Sul           0.569632177824821
Federal University of São Carlos                  0.566644488800177
Universidade Federal de Goiás                     0.557648660916931

###########################
# Life and earth sciences #
###########################

University                                        PA_F_MF   

State University of Maringá                       0.564000238251236
Universidade Federal de São Paulo                 0.505945366898768
Federal University of Pernambuco                  0.493700419972002
Rio de Janeiro State University                   0.487760416666667
Universidade Federal do Rio de Janeiro            0.485428907168038

###

Como era de se imaginar, vemos que a quantidade de mulheres autoras é maior na área de *Biomedical and health sciences* e menor em áreas 'duras' como *Mathematics and computer science* e *Physical sciences and engineering*. A **UEM**, entretanto, aparece entre as cinco universidades com mais mulheres pesquisadoras em quatro dos cinco campos do conhecimento. Fica de fora apenas em *Social sciences and humanities*.

## 9. Colaboração com a indústria

O último recorte da análise é o percentual de **artigos publicados em colaboração com entidades privadas**, excluindo hospitais e instituições de ensino (`PP_industry_collab`).

### 9.1 Análise agregada

Primeiro, como sempre, a análise de todas as áreas juntas.

In [27]:
df [ df.Field == "All sciences" ].sort_values(by = "PP_industry_collab", ascending = False).head()

,University,Country,Field,Period,Frac_counting,impact_P,P_top10,PP_top10,P_int_collab,PP_int_collab,P_industry_collab,PP_industry_collab,P_OA,PP_OA,A_F,PA_F,PA_F_MF
62765,Universidade Federal do Rio de Janeiro,Brazil,All sciences,2014–2017,1,4780.845925,311.112211,0.065075,4649.0,0.444455,423.0,0.040440,4458.0,0.426195,12191.0,0.361633,0.445920
62333,Universidade Federal de São Paulo,Brazil,All sciences,2014–2017,1,2991.923594,135.530425,0.045299,2347.0,0.357993,195.0,0.029744,2986.0,0.455461,9966.0,0.484633,0.510396
62873,Universidade Federal Fluminense,Brazil,All sciences,2014–2017,1,1575.825657,91.926940,0.058336,1304.0,0.371510,99.0,0.028205,1311.0,0.373504,3305.0,0.380936,0.420376
62225,Universidade Federal de Santa Catarina,Brazil,All sciences,2014–2017,1,2596.592015,172.235697,0.066331,2032.0,0.409595,139.0,0.028019,1383.0,0.278774,5448.0,0.387979,0.418884
62009,Universidade Federal de Goiás,Brazil,All sciences,2014–2017,1,1192.211004,63.882870,0.053584,840.0,0.320611,69.0,0.026336,992.0,0.378626,2843.0,0.408653,0.445472


A **UFRJ** aparece bem acima das demais, com 4% dos projetos. Parece, ainda assim, um valor pequeno. Vamos comparar com a média nacional:

In [28]:
df [ df.Field == "All sciences" ]["PP_industry_collab"].mean()

0.022085765114950272

A **UFRJ** é a única universidade que está significativamente acima da média nacional, que é de cerca de 2.2%.

### 9.2 Análise por campo do conhecimento

Enfim, vamos segregar os resultados por área.

In [29]:
for field in fields:
    top_5(df, field, "PP_industry_collab")

##################################
# Biomedical and health sciences #
##################################

University                                        PP_industry_collab

Universidade Federal da Bahia                     0.0381809715970821
Universidade Federal do Paraná                    0.0314188071877412
Universidade Federal de São Paulo                 0.0307389359855817
University of São Paulo                           0.0274983793730787
Universidade Federal de Santa Catarina            0.0264516129032258

###########################
# Life and earth sciences #
###########################

University                                        PP_industry_collab

Universidade Federal Fluminense                   0.0420767716535433
Universidade de Brasília                          0.0380593893768298
Federal University of Rio Grande do Norte         0.037371763556424
Universidade Federal do Rio de Janeiro            0.0358938053097345
Rio de Janeiro State University                 

A **UFRJ** novamente aparece em destaque, mas dessa vez na área de *Physical sciences and engineering*, na qual o índice de colaboração é de 6.6%. 

## 10. Principais destaques

1. A produção de *Biomedical and Health Sciences* da USP, sozinha, é superior a produção de todas as demais universidades brasileiras que constam no ranking.

- A Unicamp, no agregado de todas as ciências, tem um desempenho alto tanto na medida total de artigos publicados quanto no percentual destes que ficam entre os 10% mais citados no respectivo campo e ano.

- Quando analisamos o percentual de artigos publicados entre os 10% de maior impacto, se destacam universidades menos 'badaladas', fora do sistema estadual paulista e que não estão entre as maiores universidades federais do país. Entre elas, estão a **UFC**, **UFBA**, **Ufscar** e **UFSC**

- Quando segmentados por área do conhecimento percentuais de publicações de alto impacto, não há uma universidade que se destaque em todas as áreas. Assim, faz sentido apontar quais são as melhores **em cada campo**.

- Apesar da UFU (Universidade Federal de Uberlândia) aparecer como a universidade mais produtiva (`PP_top10`) na área de *Biomedical and health sciences*, as pesquisas de destaque que encontrei na Web of Science são na interseção do campo com a área de *Life and earth sciences*. Em compensação, a **UFBA** (Universidade Federal da Bahia), que fica em 2º por esse critério, tem um volume grande de pesquisas na área de **viroses e doenças tropicais**. 

- A área *Life and earth sciences* é a única em que as duas maiores universidades aparecem como as mais produtivas em termos percentuais (indicador `PP_top10`): **Unicamp** em 1º e **USP** em 2º.

- Os bons resultados da **UFPB**, que tem o maior indicador `PP_top10` da área de *Mathematics and computer science*, é [referendado pelo Shanghai Ranking](http://www.ufpb.br/antigo/content/matem%C3%A1tica-da-ufpb-entra-no-top-300-do-mundo).

- Na área de humanas, onde os artigos costumam ser menos citados, as publicações de maior relevância nos indicadores do Web of Science costumam estar na interesecção com outra área. No caso da UFV, por exemplo, há publicações de impacto sobre a economia da engenharia florestal. Escolhi destacar pesquisas de humanas mais "puras", como economia rural, por exemplo.

- 65.6% das publicações da área *Physical sciences and engineering* da **UERJ** são de acesso aberto. Ela é a segunda universidade com mais produções abertas no país. Nessa área específica, fica em primeiro e bastante acima das demais.

- A Universidade Estadual de Maringá (**UEM**) é a universidade com mais pesquisadores mulheres – 54% do total. No campo de *Biomedical and health sciences*, o valor chega a 63.5%. Ela também lidera na área de *Life and earth sciences* e fica em segundo em *Physical sciences and engineering*.

- **UERJ** e **UFRJ** têm altos índices de colaboração na área de *Physical sciences and engineering*. Na Web of Science, todas as produções do Instituto de Física da UFRJ aparecem como produções de ambas UFRJ e UERJ. **Isso faz algum sentido?**

## 11. Possíveis personagens

Depois de fazer a análise dos dados do Lieden Ranking, pudemos fazer buscas manuais na interface da *Web of Science* para encontrar pesquisadores representativos das universidades e departamentos que se destacaram. Eles estão listados abaixo:

### 11.1 Destaques em 'Biomedical and health sciences'

**1. Dênis Coelho de Oliveira – UFU**

[Lattes](http://buscatextual.cnpq.br/buscatextual/visualizacv.do?id=K4757061D2)

Pesquisador e professor do Instituto de Biologia da Universidade Federal de Uberlândia, que é a universidade com melhor indicador `PP_top10` para a área de *Biomedical and health sciences*. Estuda anatomia e desenvolvimento vegetal. [Seu artigo de maior repercussão](https://doi.org/10.1016/j.jinsphys.2015.11.012) é sobre galhas – inchaços semelhantes a "tumores" que aparecem no tecido de plantas – causadas por insetos. Segundo os critérios do Lieden Ranking, o paper contou 2/3 dos pontos para as áreas de *Life and earth sciences* 1/3 para a área de *Biomedical and health sciences*.


**2. Gúbio Soares Campos – UFBA**

[Lattes](http://lattes.cnpq.br/4553891428822702)

Coordenador do Laboratorio de Virologia do Instituto de Ciências da Saúde da UFBA, a 2ª universidade com maior impacto (`PP_top10`), em termos percentuais. Pesquisa, evidentemente, viroses. Em seu currículo Lattes, afirma ter encontrado as primeiras ocorrências do vírus Zika no Brasil, na cidade de Camaçari (BA), em 2015. Seu artigo de maior repercussão descreve justamente essa descoberta.

**3. Federico Costa – UFBA**

[Lattes](http://lattes.cnpq.br/9240469061681136)

Argentino, é professor e pesquisador do Instituto de Saúde Coletiva da UFBA. Suas publicações de maior destaque são revisões de pesquisa: [uma sobre leptospirose](https://journals.plos.org/plosntds/article/file?id=10.1371/journal.pntd.0003898&type=printable) e [outra sobre o vírus Zika](https://www.arca.fiocruz.br/bitstream/icict/13967/2/Weaver%20SC%20Zika%20virus%20-%20History%2c%20emergence%2c%20biology%2c%20and%20prospects%20for%20control.pdf). Em seu currículo Lattes, afirma ser pesquisador visitante nas Universidade de Yale e na Universidade de Liverpool.

### 11.2 Destaques em 'Life and earth sciences'

**1. Roseane Fett – UFSC**

[Lattes](http://lattes.cnpq.br/8462831533501212)

Fez toda a carreira acadêmica na UFSC, onde dá aulas e coordena o Programa de Pós-Graduação em Ciência dos Alimentos. Entre outros temas, pesquisa as propriedades do **mel de abelhas**. A publicação de maior impacto da UFSC na Web of Science é, justamente, [uma revisão de literatura](https://doi.org/10.1016/j.foodchem.2015.09.051) sobre esse tema.

### 11.3 Destaques em 'Mathematics and computer science'

**1. Daniel Pellegrino – UFPB**

[Lattes](http://lattes.cnpq.br/1077711232112285)

Professor da UFPB, onde fez sua graduação, coordena um grupo de pesquisa chamado "Projeto Integrado de Análise". Fez mestrado e doutorado na Unicamp. Seus dois artigos de mais impacto são [esta prova da sub-exponencialidade do raio de Bohr](http://eprints.ucm.es/29049/1/1310.2834v2.pdf) e este artigo exploratório sobre [espaços lineares](https://www-ams-org.ezproxy.cul.columbia.edu/journals/bull/2014-51-01/S0273-0979-2013-01421-6/S0273-0979-2013-01421-6.pdf).

**2. Bruno Henrique Carvalho Ribeiro – UFPB**

[Lattes](http://lattes.cnpq.br/9043204013012953)

Fez graduação e mestrado em matemática na UFPB. Seu mestrado, porém, foi na Unicamp. É diretor do departamento de matemática da UFPB.


### 11.4 Destaques em 'Physical sciences and engineering'

**1. Eduardo Bedê Barros – UFC**

[Lattes](http://lattes.cnpq.br/0233820096700167)

Pesquisador, professor e diretor do departamento de Física da UFC (Universidade Federal do Ceará). Ele fez graduação, mestrado e doutorado na instituição, além de ter estudado por um período no MIT. A UFC é a universidade com o melhor indicador `PP_top10` do Brasil, destacando-se especialmente no setor de *Physical Sciences and Engineering*.

### 11.5 Destaques em 'Social sciences and humanities'

**1. Dênis Antônio da Cunha – UFV**

[Lattes](http://lattes.cnpq.br/1033658951252242)

Fez graduação, mestrado e doutorado em Economia Aplicada na  UFV (Universidade Federal de Viçosa). Lidera um grupo de pesquisa sobre a relação da economia com os recursos ambientais e naturais. Dá aulas nos programas de pós-graduação em Economia Aplicada e Extensão Rural. O [artigo](https://www.locus.ufv.br/bitstream/handle/123456789/22800/artigo.pdf) que encontrei na Web of Science estuda o uso de sistemas de irrigação como forma de adaptação às mudanças climáticas no Brasil.

**2. João Eustáquio de Lima – UFV**

Desde 1981, é professor do departamento de economia rural da UFV. Fez a graduação em agromomia na própria universidade, onde também fez mestrado em economia rural. Depois, fez outro mestrado e doutorado na Michigan State University. Seu nome foi indicado por Dênis Antônio da Cunha, listado acima, como "o orientador mais experiente do departamento".

[Lattes](http://lattes.cnpq.br/0920624030229788)

### 11.6 Destaque em participação feminina na pesquisa

**1. Francielle Sato – UEM**

[Lattes](http://lattes.cnpq.br/7314370428188000)

Física e professora, fez toda sua carreira acadêmica na UEM. Dá aulas para graduação e pós, além de participar de grupos de pesquisa que tratam, em linhas gerais, de fármacos e biocombustível. Nesse contexto, sua espcecialide são *fenômenos fototérmicos*. Participou [deste](https://www-sciencedirect-com.ezproxy.cul.columbia.edu/science/article/pii/S0308814613019353?via%3Dihub) artigo, que está listado entre os mais citados do campo na Web of Science. Dos sete autores, cinco são mulheres.

**2. Linnyer Beatrys Ruiz Aylon – UEM**

[Lattes](http://lattes.cnpq.br/2872064505456567)

Citada em [release](http://www.noticias.uem.br/index.php?view=article&id=23776) da universidade sobre paridade de gênero, publicado quando o Lieden Ranking 2019 foi divulgado. É professora do Departamento de Informática da UEM. Ainda segundo o release, recebeu, em 2013, o prêmio IEEE Women in Engineering. Atua, especificamente, em microeletrônica. Também tem graduação em teologia (!)


### 11.7 Destaques em colaboração internacional na pesquisa

**1. Miguel Boavista Quartin – UFRJ**

[Lattes](http://lattes.cnpq.br/3080181268936724)

Graduado em Engenharia Eletrônica e doutor em Física pela UFRJ, pesquisa, resumidamente, *comsologia*. Fez pós-doutorado fora do país. Coordena quatro grupos de pesquisa. Publicou diversos artigos como [este](http://diposit.ub.edu/dspace/bitstream/2445/119262/1/670717.pdf), de alto impacto, que foi resultado da colaboração entre pesquisadores de 42 instituições, espalhadas por 15 países.